<a href="https://colab.research.google.com/github/nilesh3030/Stroke-Prediction/blob/main/Notebooks/3_Weather_data_cleaning_and_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install fastparquet

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import dateutil.parser

pd.set_option ('display.max_columns', None)
pd.set_option ('display.max_rows', None)

raw_data = pd.read_csv('/content/drive/MyDrive/Stroke_Prediction/Data/stroke_case.csv', low_memory = False)
raw_data_2021 = pd.read_csv('/content/drive/MyDrive/Stroke_Prediction/Data/stroke_data/stroke_case_2021.csv', low_memory = False)

unique_patient_address = pd.read_csv('/content/drive/MyDrive/Stroke_Prediction/Data/unique_address_with_location.csv', encoding = 'euc_kr')
unique_patient_address_2021 = pd.read_csv('/content/drive/MyDrive/Stroke_Prediction/Data/unique_address_with_location_2021.csv', encoding = 'euc_kr')
weather_data = pd.read_parquet('/content/drive/MyDrive/Stroke_Prediction/Data/Weather_Data/weather_data_2021_22_cleaned.parquet', engine = 'fastparquet')

In [ ]:
unique_station = weather_data.groupby(['Branch_name']).size().reset_index(name='Freq')

unique_station.shape

In [ ]:
unique_patient_address_2021.shape, unique_patient_address.shape

In [ ]:
unique_address = unique_patient_address.append(unique_patient_address_2021)
unique_address.shape

In [ ]:
2666+4505

In [ ]:
unique_address.head()

In [ ]:
# Dedup the data to handle the repeating values on branch datetime level
unique_address = unique_address.drop(['Unnamed: 0'], axis = 1)
unique_address = unique_address.drop_duplicates(subset=['Address'])

unique_address.shape

# Adding the location of weather station and assigning the weather station which is nearest from the patient's location

In [ ]:
weather_station_location = pd.read_excel('/content/drive/MyDrive/Stroke_Prediction/Data/Meterological_branch_and_Location.xlsx')
weather_station_location.head()

In [ ]:
## Stripping the whitespace so that join can be done easily
weather_station_location['Branch Name'] = weather_station_location['Branch Name'].str.strip()
unique_station['Branch_name'] = unique_station['Branch_name'].str.strip()

In [ ]:
## Attaching the location info to unique stations
unique_station_location = pd.merge(unique_station, weather_station_location,  how='inner', left_on=['Branch_name'], right_on = ['Branch Name'])

In [ ]:
#Shape should be same as that ensures that we have location info for all the unique weather station
unique_station_location.shape

In [ ]:
unique_station_location = unique_station_location[['Branch Name', 'Latitude', 'Longitude']]
unique_station_location.head()

In [ ]:
### Attaching the branch which is nearest from the patient's location
import numpy as np
from scipy.spatial.distance import cdist

unique_address["Branch Name"] = pd.Series([unique_station_location["Branch Name"].iloc[np.argmin(x)] for x in cdist(unique_address[["latitude", "longitude"]], unique_station_location[["Latitude", "Longitude"]])])


In [ ]:
unique_address.head()

In [ ]:
## We will create the data on 'id' level where we map the nearest branch info for each ids
id_level_data_2022 = raw_data[['jaenan_sn', 'address_si', 'address_gu', 'address_dong', 'call_d', 'call_t']]
id_level_data_2022['Address'] = id_level_data_2022['address_si'] + ", " + id_level_data_2022['address_gu'] + ", " +  id_level_data_2022['address_dong']

id_level_data_2021 = raw_data_2021[['jaenan_sn', 'address_si', 'address_gu', 'address_dong', 'call_d', 'call_t']]
id_level_data_2021['Address'] = id_level_data_2021['address_si'] + ", " + id_level_data_2021['address_gu'] + ", " +  id_level_data_2021['address_dong']

id_level_data = id_level_data_2021.append(id_level_data_2022)

print(id_level_data.shape, id_level_data_2021.shape, id_level_data_2022.shape)

In [ ]:
null_variables = round((id_level_data.isna().sum()/len(id_level_data))*100,2)
null_variables

In [ ]:
(123152*(100-8.84))/100

In [ ]:
nearest_branch_super = id_level_data.merge(unique_address, on = 'Address')
print(nearest_branch_super.shape)

#Processing the Nearest Branch data

In [ ]:
print(nearest_branch_super.shape)
nearest_branch_super.head()

In [ ]:
nearest_branch_super.columns

In [ ]:
vars = ['jaenan_sn','call_d', 'call_t', 'latitude', 'longitude', 'Branch Name']
nearest_branch = nearest_branch_super[vars]
#rename the branch name to weather station######
#'latitude', 'longitude' are the locations for the address i.e. patient location
# 'Latitude', 'Longitude' are location respective to branch address

In [ ]:
convert_dict = {'call_d': str,
                'call_t': str}
 
nearest_branch = nearest_branch.astype(convert_dict)

nearest_branch["Datetime_patient"] = nearest_branch["call_d"] + nearest_branch["call_t"]

In [ ]:
def convert_datetime(row):
  try:
    if len(row['Datetime_patient']) == 11 or len(row['Datetime_patient']) == 12:
      return(pd.to_datetime(row['Datetime_patient'], format = '%Y%m%d%H%M'))
  except: 
    return(np.nan)


nearest_branch['Datetime_converted'] = nearest_branch.apply(lambda row: convert_datetime(row), axis=1)

In [ ]:
nearest_branch['Datetime_converted'] = nearest_branch['Datetime_converted'].dt.strftime('%Y-%m-%d %H')

In [ ]:
print(nearest_branch.shape)
nearest_branch.head()

#Combining weather and nearest-branch data

In [ ]:
new_df = pd.merge(nearest_branch, weather_data,  how='left', left_on=['Branch Name','Datetime_converted'], right_on = ['Branch_name','Date_time'])
print(new_df.shape)
new_df.head()

In [ ]:
null_variables = round((new_df.isna().sum()/len(new_df))*100,2)
null_variables